In [1]:
import pandas as pd


In [2]:
path='/content/drive/MyDrive/Twitter_Data.csv'

In [3]:
twitter_df=pd.read_csv(path)

In [4]:
path1='/content/drive/MyDrive/Reddit_Data.csv'

In [5]:
reddit_df=pd.read_csv(path1)

In [6]:
# Display basic information about the Twitter dataset
print(twitter_df.info())

# Display basic information about the Reddit dataset
print(reddit_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB
None


In [7]:
# Handle missing values
twitter_df = twitter_df.dropna()
reddit_df = reddit_df.dropna()

# Lowercase the text
twitter_df['cleaned_tweets'] = twitter_df['clean_text'].str.lower()
reddit_df['cleaned_comments'] = reddit_df['clean_comment'].str.lower()

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize Twitter text
tokenizer_twitter = Tokenizer()
tokenizer_twitter.fit_on_texts(twitter_df['cleaned_tweets'])
sequences_twitter = tokenizer_twitter.texts_to_sequences(twitter_df['cleaned_tweets'])
padded_sequences_twitter = pad_sequences(sequences_twitter, maxlen=250, padding='post')

# Tokenize Reddit text
tokenizer_reddit = Tokenizer()
tokenizer_reddit.fit_on_texts(reddit_df['cleaned_comments'])
sequences_reddit = tokenizer_reddit.texts_to_sequences(reddit_df['cleaned_comments'])
padded_sequences_reddit = pad_sequences(sequences_reddit, maxlen=250, padding='post')


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer_reddit.word_index) + 1, output_dim=100, input_length=padded_sequences_reddit.shape[1]))
model.add(LSTM(20))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'],run_eagerly=True)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5472000   
                                                                 
 lstm (LSTM)                 (None, 20)                9680      
                                                                 
 dense (Dense)               (None, 3)                 63        
                                                                 
Total params: 5481743 (20.91 MB)
Trainable params: 5481743 (20.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model1 = Sequential()
model1.add(Embedding(input_dim=len(tokenizer_twitter.word_index) + 1, output_dim=100, input_length=padded_sequences_twitter.shape[1]))
model1.add(LSTM(20))
model1.add(Dense(3, activation='softmax'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'],run_eagerly=True)
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 100)          11367900  
                                                                 
 lstm_1 (LSTM)               (None, 20)                9680      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 11377643 (43.40 MB)
Trainable params: 11377643 (43.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
# Assuming 'sentiment_label' is the column containing sentiment labels

# Convert labels to categorical format
labels_reddit = to_categorical(reddit_df['category'],num_classes=3)

# Train-test split for Reddit data
X_train_reddit, X_test_reddit, y_train_reddit, y_test_reddit = train_test_split(
    padded_sequences_reddit, labels_reddit, test_size=0.2, random_state=42
)
# Further split the training set into training and validation sets
X_train_reddit, X_val_reddit, y_train_reddit, y_val_reddit = train_test_split(
    X_train_reddit, y_train_reddit, test_size=0.2, random_state=42
)
import numpy as np
print(X_train_reddit.shape, y_train_reddit.shape)
print(X_val_reddit.shape, y_val_reddit.shape)

print(np.any(np.isnan(X_train_reddit)))
print(np.any(np.isnan(y_train_reddit)))

print(X_train_reddit[:5])
print(y_train_reddit[:5])

# Train the model on Reddit data
model.fit(X_train_reddit, y_train_reddit, epochs=5, batch_size=10, validation_data=(X_val_reddit, y_val_reddit))


(23775, 250) (23775, 3)
(5944, 250) (5944, 3)
False
False
[[    6     7   603 ...     0     0     0]
 [ 1152   858 32909 ...     0     0     0]
 [   20    19     8 ...     0     0     0]
 [   75    70     2 ...     0     0     0]
 [ 1620  4317    17 ...     0     0     0]]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]
Epoch 1/5
2378/2378 [==============================] - 2363s 994ms/step - loss: 1.0670 - accuracy: 0.4244 - val_loss: 1.0627 - val_accuracy: 0.4241
Epoch 2/5
2378/2378 [==============================] - 2295s 965ms/step - loss: 1.0089 - accuracy: 0.5092 - val_loss: 0.9757 - val_accuracy: 0.5553
Epoch 3/5
2378/2378 [==============================] - 2326s 978ms/step - loss: 0.9929 - accuracy: 0.5285 - val_loss: 0.9536 - val_accuracy: 0.5616
Epoch 4/5
2378/2378 [==============================] - 2398s 1s/step - loss: 0.8746 - accuracy: 0.6122 - val_loss: 0.7636 - val_accuracy: 0.6699
Epoch 5/5
2378/2378 [==============================] - 2350s 988ms/step - lo

In [ ]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
# Assuming 'sentiment_label' is the column containing sentiment labels

# Convert labels to categorical format
labels_twitter = to_categorical(twitter_df['category'],num_classes=3)

# Train-test split for Twitter data
X_train_twitter, X_test_twitter, y_train_twitter, y_test_twitter= train_test_split(
    padded_sequences_twitter, labels_twitter, test_size=0.2, random_state=42
)
# Further split the training set into training and validation sets
X_train_twitter, X_val_twitter, y_train_twitter, y_val_twitter = train_test_split(
    X_train_twitter, y_train_twitter, test_size=0.2, random_state=42
)
import numpy as np
print(X_train_twitter.shape, y_train_twitter.shape)
print(X_val_twitter.shape, y_val_twitter.shape)

print(np.any(np.isnan(X_train_twitter)))
print(np.any(np.isnan(y_train_twitter)))

print(X_train_twitter[:5])
print(y_train_twitter[:5])

# Train the model on Twitter data
model1.fit(X_train_twitter, y_train_twitter, epochs=5, batch_size=13, validation_data=(X_val_twitter, y_val_twitter))

(104300, 250) (104300, 3)
(26075, 250) (26075, 3)
False
False
[[64414    16    10 ...     0     0     0]
 [  201   107  1934 ...     0     0     0]
 [  126  1931  1272 ...     0     0     0]
 [14848  8967  1334 ...     0     0     0]
 [  186    35  1962 ...     0     0     0]]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Epoch 1/5
8024/8024 [==============================] - 10974s 1s/step - loss: 1.0599 - accuracy: 0.4430 - val_loss: 1.0589 - val_accuracy: 0.4472
Epoch 2/5
8024/8024 [==============================] - 11414s 1s/step - loss: 1.0594 - accuracy: 0.4431 - val_loss: 1.0590 - val_accuracy: 0.4472
Epoch 3/5
6795/8024 [========================>.....] - ETA: 26:05 - loss: 1.0595 - accuracy: 0.4425

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Assuming 'model' is your trained model and 'embedding_layer' is the Embedding layer in your model

# Extract the weights of the embedding layer
embedding_weights = model.get_layer('embedding_2').get_weights()[0]

# Define a subset of the embedding weights (optional: you can use all weights)
subset_embedding_weights = embedding_weights[:1000]  # Adjust the subset size based on your preference

# Apply t-SNE to reduce the dimensionality of the embeddings
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(subset_embedding_weights)

# Visualize the embeddings for all sentiments
plt.figure(figsize=(12, 10))

# Plot all embeddings
plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], alpha=0.5, label='All')

# Annotate a few points with words for better understanding
for i in range(len(subset_embedding_weights)):
    plt.annotate(i, (embeddings_tsne[i, 0], embeddings_tsne[i, 1]))

# Visualize the embeddings for positive sentiment (adjust indices accordingly)
# positive_indices = [i for i in range(len(subset_embedding_weights)) if y_test_reddit[i, 1] == 1]
# plt.scatter(embeddings_tsne[positive_indices, 0], embeddings_tsne[positive_indices, 1], alpha=0.5, label='Positive', color='green')

# Visualize the embeddings for negative sentiment (adjust indices accordingly)
# negative_indices = [i for i in range(len(subset_embedding_weights)) if y_test_reddit[i, 0] == 1]
# plt.scatter(embeddings_tsne[negative_indices, 0], embeddings_tsne[negative_indices, 1], alpha=0.5, label='Negative', color='red')

plt.title('t-SNE Visualization of Word Embeddings')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Assuming 'model' is your trained model and 'embedding_layer' is the Embedding layer in your model

# Extract the weights of the embedding layer
embedding_weights = model.get_layer('embedding_2').get_weights()[0]

# Define a subset of the embedding weights (optional: you can use all weights)
subset_embedding_weights = embedding_weights[:1000]  # Adjust the subset size based on your preference

# Apply t-SNE to reduce the dimensionality of the embeddings
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(subset_embedding_weights)

# Visualize the embeddings for all sentiments
plt.figure(figsize=(12, 10))

# Plot all embeddings
# plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], alpha=0.5, label='All')

# Annotate a few points with words for better understanding
for i in range(len(subset_embedding_weights)):
    plt.annotate(i, (embeddings_tsne[i, 0], embeddings_tsne[i, 1]))

# Visualize the embeddings for positive sentiment (adjust indices accordingly)
positive_indices = [i for i in range(len(subset_embedding_weights)) if y_test_reddit[i, 1] == 1]
plt.scatter(embeddings_tsne[positive_indices, 0], embeddings_tsne[positive_indices, 1], alpha=0.5, label='Positive', color='green')

# Visualize the embeddings for negative sentiment (adjust indices accordingly)
# negative_indices = [i for i in range(len(subset_embedding_weights)) if y_test_reddit[i, 0] == 1]
# plt.scatter(embeddings_tsne[negative_indices, 0], embeddings_tsne[negative_indices, 1], alpha=0.5, label='Negative', color='red')

plt.title('t-SNE Visualization of Word Embeddings')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Assuming 'model' is your trained model and 'embedding_layer' is the Embedding layer in your model

# Extract the weights of the embedding layer
embedding_weights = model.get_layer('embedding_2').get_weights()[0]

# Define a subset of the embedding weights (optional: you can use all weights)
subset_embedding_weights = embedding_weights[:1000]  # Adjust the subset size based on your preference

# Apply t-SNE to reduce the dimensionality of the embeddings
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(subset_embedding_weights)

# Visualize the embeddings for all sentiments
plt.figure(figsize=(12, 10))

# Plot all embeddings
# plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], alpha=0.5, label='All')

# Annotate a few points with words for better understanding
for i in range(len(subset_embedding_weights)):
    plt.annotate(i, (embeddings_tsne[i, 0], embeddings_tsne[i, 1]))

# Visualize the embeddings for positive sentiment (adjust indices accordingly)
# positive_indices = [i for i in range(len(subset_embedding_weights)) if y_test_reddit[i, 1] == 1]
# plt.scatter(embeddings_tsne[positive_indices, 0], embeddings_tsne[positive_indices, 1], alpha=0.5, label='Positive', color='green')

# Visualize the embeddings for negative sentiment (adjust indices accordingly)
negative_indices = [i for i in range(len(subset_embedding_weights)) if y_test_reddit[i, 0] == 1]
plt.scatter(embeddings_tsne[negative_indices, 0], embeddings_tsne[negative_indices, 1], alpha=0.5, label='Negative', color='red')

plt.title('t-SNE Visualization of Word Embeddings')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Assuming 'model1' is your trained model and 'embedding_layer' is the Embedding layer in your model

# Extract the weights of the embedding layer
embedding_weights = model1.get_layer('embedding_2').get_weights()[0]

# Define a subset of the embedding weights (optional: you can use all weights)
subset_embedding_weights = embedding_weights[:1000]  # Adjust the subset size based on your preference

# Apply t-SNE to reduce the dimensionality of the embeddings
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(subset_embedding_weights)

# Visualize the embeddings for all sentiments
plt.figure(figsize=(12, 10))

# Plot all embeddings
plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], alpha=0.5, label='All')

# Annotate a few points with words for better understanding
for i in range(len(subset_embedding_weights)):
    plt.annotate(i, (embeddings_tsne[i, 0], embeddings_tsne[i, 1]))

# Visualize the embeddings for positive sentiment (adjust indices accordingly)
positive_indices = [i for i in range(len(subset_embedding_weights)) if y_test_reddit[i, 1] == 1]
plt.scatter(embeddings_tsne[positive_indices, 0], embeddings_tsne[positive_indices, 1], alpha=0.5, label='Positive', color='green')

# Visualize the embeddings for negative sentiment (adjust indices accordingly)
negative_indices = [i for i in range(len(subset_embedding_weights)) if y_test_reddit[i, 0] == 1]
plt.scatter(embeddings_tsne[negative_indices, 0], embeddings_tsne[negative_indices, 1], alpha=0.5, label='Negative', color='red')

plt.title('t-SNE Visualization of Word Embeddings')
plt.legend()
plt.show()
